In [5]:
import sys
sys.path.append("../")

import subprocess
import datetime
from joblib import Parallel, delayed

import json
import os

import matplotlib.pyplot as plt
from tqdm import tqdm
import optuna

from run import *

In [7]:
start_case = 0
case_num = 1000

cases = [
    (f"tools/in/{seed:04}.txt", f"tools/out/{seed:04}.txt")
    for seed in range(start_case, start_case + case_num)
]


def objective(trial: optuna.trial.Trial) -> float:
    # min_k = trial.suggest_float("max_k", low=1.5, high=4.5)
    # max_k = trial.suggest_float("min_k", low=5.0, high=7.0)
    # k_p = trial.suggest_float("k_p", low=0.5, high=2.0)
    min_k = 4
    max_k = 5.5
    k_p = 0.85

    start_step = trial.suggest_float("start_step", low=0.2, high=1.5)
    end_step = trial.suggest_float("end_step", low=max(1.2, start_step + 0.1), high=3.2)
    step_cnt = trial.suggest_int("step_cnt", low=2, high=12)

    args = " ".join(map(str, [min_k, max_k, k_p, start_step, end_step, step_cnt]))

    solver_path = "./target/release/ahc030"
    solver_cmd = f"./tools/target/release/tester {solver_path} {args}"
    solver_version = f"optuna-now-{args}"
    runner = Runner(
        Input,
        Result,
        solver_cmd=solver_cmd,
        solver_version=solver_version,
        database_csv="../log/database.csv",
        log_file="../log/a.log",
        verbose=0,
    )
    runner.run(cases=cases, verbose=0)
    score_df = runner.evaluate_overall_relative_score(
        ignore_solver_prefix="optuna-now-"
    )
    return score_df.relative_score.mean()


subprocess.run(
    "cargo build --features local --release --manifest-path=../Cargo.toml", shell=True
)

study = optuna.create_study(
    direction="maximize",
    storage="sqlite:///ahc030.db",
    study_name="tune-steps-03",
    load_if_exists=True,
)
print("start optimize")
study.enqueue_trial(
    {
        "min_k": 4.0,
        "max_k": 5.5,
        "k_p": 0.85,
        "start_step": 0.8,
        "end_step": 2.0,
        "step_cnt": 7,
    }
)
study.optimize(objective, n_trials=300, show_progress_bar=True)
study.best_params

    Finished release [optimized] target(s) in 0.05s
[I 2024-02-19 01:22:22,441] Using an existing study with name 'tune-steps-03' instead of creating a new one.


start optimize


/Users/tatsuyaishii/.pyenv/versions/3.10.4/lib/python3.10/site-packages/optuna/progress_bar.py:49: ExperimentalWarning: Progress bar is experimental (supported from v1.2.0). The interface can change in the future.
  self._init_valid()


  0%|          | 0/300 [00:00<?, ?it/s]

[W 2024-02-19 01:23:51,074] Trial 2 failed because of the following error: KeyboardInterrupt()
Traceback (most recent call last):
  File "/Users/tatsuyaishii/.pyenv/versions/3.10.4/lib/python3.10/site-packages/optuna/study/_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "/var/folders/3n/z49mbd414cx0_x991z2fyqjc0000gn/T/ipykernel_19556/3802965821.py", line 36, in objective
    runner.run(cases=cases, verbose=0)
  File "/Users/tatsuyaishii/dev/heuristics-contests/ahc030/notebooks/../run.py", line 64, in run
    results = Parallel(n_jobs=-1, verbose=verbose)(
  File "/Users/tatsuyaishii/.pyenv/versions/3.10.4/lib/python3.10/site-packages/joblib/parallel.py", line 1056, in __call__
    self.retrieve()
  File "/Users/tatsuyaishii/.pyenv/versions/3.10.4/lib/python3.10/site-packages/joblib/parallel.py", line 935, in retrieve
    self._output.extend(job.get(timeout=self.timeout))
  File "/Users/tatsuyaishii/.pyenv/versions/3.10.4/lib/python3.10/site-packages/job

KeyboardInterrupt: 

In [ ]:
optuna.visualization.plot_slice(study)

In [ ]:
optuna.visualization.plot_optimization_history(study)